In [ ]:
import numpy as np
import pandas as pd
import os
import shutil
import pickle
import glob
from tqdm import tqdm
import pydicom
import SimpleITK as sitk
import zipfile
import pydicom

In [ ]:
with open('../input/rsna-series-labels/first_section_series.pkl', 'rb') as f:
    x1 = pickle.load(f)

with open('../input/rsna-series-labels/middle_section_series.pkl', 'rb') as f:
    x2 = pickle.load(f)
    
with open('../input/rsna-series-labels/last_section_series.pkl', 'rb') as f:
    x3 = pickle.load(f)

In [ ]:
selected = pd.read_csv('../input/extract-data-for-three-labelers/labeler1.csv')
selected.head()

In [ ]:
print("Epidural: ",
      len(selected.loc[selected['Epidural']==1]),
      ', Ratio: ',
      str(len(selected.loc[selected['Epidural']==1])/len(selected))[0:6])

print("Intraparenchymal: ",
      len(selected.loc[selected['Intraparenchymal']==1]),
      ', Ratio: ',
      str(len(selected.loc[selected['Intraparenchymal']==1])/len(selected))[0:6])

print("Intraventricular: ",
      len(selected.loc[selected['Intraventricular']==1]),
      ', Ratio: ',
      str(len(selected.loc[selected['Intraventricular']==1])/len(selected))[0:6])

print("Subarachnoid: ",
      len(selected.loc[selected['Subarachnoid']==1]),
      ', Ratio: ',
      str(len(selected.loc[selected['Subarachnoid']==1])/len(selected))[0:6])

print("Subdural: ",
      len(selected.loc[selected['Subdural']==1]),
      ', Ratio: ',
      str(len(selected.loc[selected['Subdural']==1])/len(selected))[0:6])

In [ ]:
def convert(dicoms):
    image = []
    for i in dicoms:
        img = sitk.ReadImage(i)
        img = sitk.GetArrayFromImage(img)[0]
        image.append(img)
    image = np.array(image)
    image = sitk.GetImageFromArray(image)
    return image


print('creating archive')
zf = zipfile.ZipFile('Labeler1_Dataset.zip', mode='w')

for j in tqdm(range(len(selected))):
    idd = selected['UID'].values[j]
    imglist = x1[idd]
    imglist.extend(x2[idd])
    imglist.extend(x3[idd])
    dicom_path = imglist
    image = convert(dicom_path)
    sitk.WriteImage(image, idd+'.nii.gz')
    zf.write(idd+'.nii.gz', idd+'.nii.gz', zipfile.ZIP_DEFLATED)
    os.remove(idd+'.nii.gz')

In [ ]:
midline = pd.read_csv('../input/cq500-midline-shift-labels/MidlineShift.csv')
midline = midline.loc[midline['MidlineShift']==1]
midline = midline['name'].values
print(len(midline))
cq500 = glob.glob('../input/cq500-hemorrhage-dicom/CQ500_Images/*')
cq500.extend(glob.glob('../input/cq500-nonhemorrhage-dicom/CQ500_NonHemo_Images/*'))
cq500 = [x for x in cq500 if x.split('/')[-1] in midline]
print(len(cq500))

cq500_series = []
for i in tqdm(range(len(cq500))):
    series = glob.glob(cq500[i]+'/*')
    for j in range(len(series)):
        dicoms = glob.glob(series[j]+'/*.dcm')
        if (len(dicoms) <= 50):
            cq500_series.append(series[j])
print(len(cq500_series))

def convert_series(path):
    series_id = sitk.ImageSeriesReader.GetGDCMSeriesIDs(path)
    sid = series_id[0]
    reader = sitk.ImageSeriesReader()
    image = sitk.ReadImage(reader.GetGDCMSeriesFileNames(path, sid))
    return image

for j in tqdm(range(0, 22)):
    image = convert_series(cq500_series[j])
    sitk.WriteImage(image, 'CQ500_{}.nii.gz'.format(j))
    zf.write('CQ500_{}.nii.gz'.format(j), 'CQ500_{}.nii.gz'.format(j), zipfile.ZIP_DEFLATED)
    os.remove('CQ500_{}.nii.gz'.format(j))
    
zf.close()